In [1]:
"""Capacitated Vehicle Routing Problem with Time Windows (CVRPTW).
"""
from __future__ import print_function
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

In [2]:
def compute_revisits(data, dict_names, _distances, visits, demands):
  """Converts node number to take into account revisiting."""
  max_num = data["num_unique_locations"]
  revisits = []
  if data["num_locations"] == data["num_unique_locations"]:
    pass
  else:
    for node_index, number_of_visits in enumerate(visits):
      if number_of_visits > 1:
        revisits_list = [node_index]
        for x in range(number_of_visits - 1):
          dict_names[max_num] = dict_names[node_index]
          _distances.append(_distances[node_index])
          for distance_list in _distances:
            distance_list.append(distance_list[node_index])
          demands.append(demands[node_index])
          revisits_list.append(max_num)
          max_num += 1
        if number_of_visits == 2:
          revisits.append(revisits_list)
        elif number_of_visits == 3:
          revisits.append([revisits_list[0], revisits_list[1]])
          revisits.append([revisits_list[0], revisits_list[2]])
          revisits.append([revisits_list[1], revisits_list[2]])
        elif number_of_visits == 4:
          revisits.append([revisits_list[0], revisits_list[1]])
          revisits.append([revisits_list[0], revisits_list[2]])
          revisits.append([revisits_list[0], revisits_list[3]])
          revisits.append([revisits_list[1], revisits_list[2]])
          revisits.append([revisits_list[1], revisits_list[3]])
          revisits.append([revisits_list[2], revisits_list[3]])
        elif number_of_visits == 5:
          revisits.append([revisits_list[0], revisits_list[1]])
          revisits.append([revisits_list[0], revisits_list[2]])
          revisits.append([revisits_list[0], revisits_list[3]])
          revisits.append([revisits_list[0], revisits_list[4]])
          revisits.append([revisits_list[1], revisits_list[2]])
          revisits.append([revisits_list[1], revisits_list[3]])
          revisits.append([revisits_list[1], revisits_list[4]])
          revisits.append([revisits_list[2], revisits_list[3]])
          revisits.append([revisits_list[2], revisits_list[4]])
          revisits.append([revisits_list[3], revisits_list[4]])
  #print(revisits, dict_names, _distances, demands)
  return revisits, dict_names, _distances, demands

In [3]:
###########################
# Problem Data Definition #
###########################
def create_data_model():
  """Creates the data for the example."""
  data = {}
  # Array of distances between locations.
  _distances = \
          [
           [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
           [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
           [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
           [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
           [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
           [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
           [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
           [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
           [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
           [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
           [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
           [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
           [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
           [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
           [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
           [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
           [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0]
          ]

  demands = [0, 1, 1, 2, 1, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]

  visits = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3]
  
  dict_names = {0: "Home", 1: "PDV_1", 2: "PDV_2", 3: "PDV_3", 4: "PDV_4", 5: "PDV_5", 6: "PDV_6",
                7: "PDV_7", 8: "PDV_8", 9: "PDV_9", 10: "PDV_10", 11: "PDV_11", 12: "PDV_12",
                13: "PDV_13", 14: "PDV_14", 15: "PDV_15", 16: "PDV_16"}

  
  data["num_unique_locations"] = len(_distances)
  data["num_locations"] = sum(visits)
  data["num_vehicles"] = 4
  data["depot"] = [0, 0, 0, 0]
  data["arrive_location"] = data["depot"]
  data["time_per_demand_unit"] = 5
  data["vehicle_speed"] = 83.33
  data["horizon"] = 50
  data["penalty"] = 1000000
  
  revisits, dict_names, _distances, demands = compute_revisits(data, dict_names, _distances, visits, demands)
  
  data["revisits"] = revisits
  data["demands"] = demands
  data["distances"] = _distances
  data["dict_names"] = dict_names
  return data

In [4]:
#######################
# Problem Constraints #
#######################
def create_distance_callback(data):
  """Creates callback to return distance between points."""
  distances = data["distances"]

  def distance_callback(from_node, to_node):
    """Returns the manhattan distance between the two nodes"""
    return distances[from_node][to_node]

  return distance_callback

def create_time_callback(data):
  """Creates callback to get total times between locations."""
  def service_time(node):
    """Gets the service time for the specified location."""
    return data["demands"][node] * data["time_per_demand_unit"]

  def travel_time(from_node, to_node):
    """Gets the travel times between two locations."""
    travel_time = data["distances"][from_node][to_node] / data["vehicle_speed"]
    return travel_time

  def time_callback(from_node, to_node):
    """Returns the total time between the two nodes"""
    serv_time = service_time(from_node)
    trav_time = travel_time(from_node, to_node)
    return serv_time + trav_time

  return time_callback

def add_time_constraints(routing, data, time_callback):
  """Add time constraints."""
  time = "Time"
  routing.AddDimension(
    time_callback,
    data["horizon"], # allow waiting time
    data["horizon"], # maximum time per vehicle
    False, # Don't force start cumul to zero. This doesn't have any effect in this example,
           # since the depot has a start window of (0, 0).
    time)

def add_revisit_constraints(routing, data):
  """Add revisit constraints."""
  for pair_revisit in data["revisits"]:
    constraintActive = routing.ActiveVar(routing.NodeToIndex(pair_revisit[0])) * routing.ActiveVar(routing.NodeToIndex(pair_revisit[1]))
    routing.solver().Add(
    constraintActive * routing.VehicleVar(routing.NodeToIndex(pair_revisit[0])) != routing.VehicleVar(routing.NodeToIndex(pair_revisit[1])))

In [5]:
###########
# Printer #
###########
def print_solution(data, routing, assignment):
  """Prints assignment on console"""
  # Inspect solution.
  time_dimension = routing.GetDimensionOrDie('Time')
  total_dist = 0
  time_matrix = 0
  visited_locations = []
  dropped_locations = list(range(data["num_locations"]))

  for vehicle_id in range(data["num_vehicles"]):
    index = routing.Start(vehicle_id)
    plan_output = 'Route for vehicle {0}:\n'.format(vehicle_id)
    route_dist = 0
    while not routing.IsEnd(index):
      node_index = routing.IndexToNode(index)
      next_node_index = routing.IndexToNode(
        assignment.Value(routing.NextVar(index)))
      visited_locations.append(next_node_index)
      if next_node_index in dropped_locations:
        dropped_locations.remove(next_node_index)
      route_dist += routing.GetArcCostForVehicle(node_index, next_node_index, vehicle_id)
      time_var = time_dimension.CumulVar(index)
      time_min = assignment.Min(time_var)
      time_max = assignment.Max(time_var)
      plan_output += ' {0} Time({1},{2}) ->'.format(data["dict_names"][node_index], time_min, time_max)
      index = assignment.Value(routing.NextVar(index))

    node_index = routing.IndexToNode(index)
    time_var = time_dimension.CumulVar(index)
    route_time = assignment.Value(time_var)
    time_min = assignment.Min(time_var)
    time_max = assignment.Max(time_var)
    total_dist += route_dist
    time_matrix += route_time
    plan_output += ' {0} Time({1},{2})\n'.format(data["dict_names"][node_index], time_min, time_max)
    plan_output += 'Distance of the route: {0} m\n'.format(route_dist)
    plan_output += 'Time of the route: {0} min\n'.format(route_time)
    print(plan_output)
  print('Total Distance of all routes: {0} m'.format(total_dist))
  print('Total Time of all routes: {0} min'.format(time_matrix))
  dropped_visits_list = list(set(range(data["num_locations"])) - set(visited_locations))
  #print('Dropped visits: ', list(set(range(data["num_locations"])) - set(visited_locations)))
  print('Dropped visits: ', list(map(data["dict_names"].get, dropped_visits_list)))

In [6]:
########
# Main #
########
def main():
  """Entry point of the program"""
  data = create_data_model()

  # Create Routing Model
  routing = pywrapcp.RoutingModel(data["num_locations"], data["num_vehicles"], 
                                  data["depot"], data["arrive_location"])
  # Define weight of each edge
  distance_callback = create_distance_callback(data)
  routing.SetArcCostEvaluatorOfAllVehicles(distance_callback)
  # Add Time constraint
  time_callback = create_time_callback(data)
  add_time_constraints(routing, data, time_callback)
  # Add Revisit constraint
  add_revisit_constraints(routing, data)
  # Setting first solution heuristic (cheapest addition).
  search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
  search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
  # Adding penalty costs to allow dropping visits.
  for i in range(1, data["num_locations"]):
    routing.AddDisjunction([routing.NodeToIndex(i)], data["penalty"])
    
  # Solve the problem.
  assignment = routing.SolveWithParameters(search_parameters)
  if assignment:
    printer = print_solution(data, routing, assignment)

In [7]:
%%time
main()

Route for vehicle 0:
 Home Time(0,2) -> PDV_13 Time(4,6) -> PDV_12 Time(26,28) -> PDV_11 Time(37,39) -> Home Time(48,50)
Distance of the route: 1164 m
Time of the route: 48 min

Route for vehicle 1:
 Home Time(0,4) -> PDV_8 Time(3,7) -> Home Time(46,50)
Distance of the route: 616 m
Time of the route: 46 min

Route for vehicle 2:
 Home Time(0,6) -> PDV_7 Time(2,8) -> Home Time(44,50)
Distance of the route: 388 m
Time of the route: 44 min

Route for vehicle 3:
 Home Time(0,3) -> PDV_9 Time(2,5) -> PDV_5 Time(9,12) -> PDV_6 Time(21,24) -> Home Time(47,50)
Distance of the route: 1164 m
Time of the route: 47 min

Total Distance of all routes: 3332 m
Total Time of all routes: 185 min
Dropped visits:  ['PDV_1', 'PDV_2', 'PDV_3', 'PDV_4', 'PDV_10', 'PDV_14', 'PDV_15', 'PDV_16', 'PDV_16', 'PDV_16']
Wall time: 103 ms
